In [ ]:
109409004經濟三趙庭暄_1111機器學習概論期中作業
    我選擇參加kaggle的House Prices - Advanced Regression Techniques競賽
    
    並獲得了5.14381的分數。在2023/4/10位於leaderbroad的第4346名
    
    https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/leaderboard#

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import statsmodels.api as sm

In [ ]:
基本的import dataset
我玩了一下neighborhood等等的幾個變數分析

In [ ]:
#import dataset
df = pd.read_csv("data/train.csv")
#observing dataset
df.head(10)
df.info()
df.describe()
df['Neighborhood'].value_counts()
sns.distplot(df['SalePrice'])

In [ ]:
一開始我不曉得要使用甚麼方式來建構模型比較好，於是我先看看上次上課內容大概一開始都如何分析數據、進一步了解各項變數之間的關係。
也隨意選擇兩或四個變數，看看他們的關係，並且在這個過程中慢慢摸索資料的各個特徵及相關變數都有些甚麼。
我也上網搜尋找到幾個不一樣的分析方式，例如熱力圖就蠻有趣的。我也做了一些基本款，像是scatterplot及pairplot，還有correlation。
雖然這個過程一開始乍看只是無頭蒼蠅的亂晃，隨意找幾個有興趣的分析方式，好像對整個作業並沒有甚麼幫助，但熱力圖及correlartion啟發了我。

我原本在網路上看到熱力圖的時候覺得這會是個很酷很有幫助的表格，實際上畫出來之後他因為變數數距實在是太多了
根本不曉得那個小小的格子代表的是誰跟誰，所以我決定排除更多項目

In [ ]:
#使用scatterplot觀察兩變數的關係
sns.scatterplot(x='GrLivArea', y='SalePrice', data=df)
sns.scatterplot(x='TotalBsmtSF', y='SalePrice', data=df)
sns.scatterplot(x='YearBuilt', y='SalePrice', data=df)

#使用boxplot觀察兩變數的關係
sns.boxplot(x='OverallQual', y='SalePrice', data=df)
sns.boxplot(x='Neighborhood', y='SalePrice', data=df)

#觀察correlation
corr_matrix = df.corr()
print(df.corr())

#使用heatmap觀察所有變數的關係
plt.figure(figsize=(12, 9))
sns.heatmap(corr_matrix, cmap='coolwarm', annot=False, square=True)

#使用pairplot觀察四變數的關係
sns.pairplot(df, vars=["SalePrice", "OverallQual", "GrLivArea", "GarageCars"])
sns.pairplot(df)

In [ ]:
接下來就是修正數據，先從檢查各項目的missing values有多少開始

In [ ]:
#prepare to train model!
#檢查missing values
missingV = df.isnull().sum()
print("Total missing values in the dataset: ", missingV.sum())
print("\nColumns with missing values:\n", missingV[missingV > 0])

In [ ]:
再來就是drop一些不需要、用不到的數據
這裡也有運用到前面的bar圖來做確認
以及填補只有缺漏一些些的項目

這邊很麻煩的是有些項目不是數值，所以我使用眾數跟中位數分別填補
後來仔細想想，覺得自己真的是太蠢了哈哈哈
我就應該要先把每個項目都變成數值化，再來填補缺漏的啊

In [ ]:
#確定missing values在哪個變數最多，然後丟掉他們
missing = missingV[missingV > 0]
missing.sort_values(inplace=True)
missing.plot.bar()

#根據bar圖看到這五項卻露太多了，drop他們
df = df.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'], axis=1)

#Electrical只有一項缺漏
df = df.dropna(subset=['Electrical'])

#其餘的缺漏值fillna
#考慮變數的資料型態及分佈狀況
#若obj，則填入眾數mode；int，則填入中位數median
df.loc[:, 'LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].median())
df.loc[:, 'GarageType'] = df['GarageType'].fillna(df['GarageType'].mode()[0])
df.loc[:, 'GarageYrBlt'] = df['GarageYrBlt'].fillna(df['GarageYrBlt'].median())
df.loc[:, 'GarageFinish'] = df['GarageFinish'].fillna(df['GarageFinish'].mode()[0])
df.loc[:, 'GarageQual'] = df['GarageQual'].fillna(df['GarageQual'].mode()[0])
df.loc[:, 'GarageCond'] = df['GarageCond'].fillna(df['GarageCond'].mode()[0])
df.loc[:, 'BsmtQual'] = df['BsmtQual'].fillna(df['BsmtQual'].mode()[0])
df.loc[:, 'BsmtCond'] = df['BsmtCond'].fillna(df['BsmtCond'].mode()[0])
df.loc[:, 'BsmtExposure'] = df['BsmtExposure'].fillna(df['BsmtExposure'].mode()[0])
df.loc[:, 'BsmtFinType1'] = df['BsmtFinType1'].fillna(df['BsmtFinType1'].mode()[0])
df.loc[:, 'BsmtFinType2'] = df['BsmtFinType2'].fillna(df['BsmtFinType2'].mode()[0])
df.loc[:, 'MasVnrType'] = df['MasVnrType'].fillna(df['MasVnrType'].mode()[0])
df.loc[:, 'MasVnrArea'] = df['MasVnrArea'].fillna(df['MasVnrArea'].median())
df.info()
id_nums = df.loc[:, 'Id']
print(id_nums)

In [ ]:
該數據化的項目都數據化之後，做到這裡的時候也差不多要來準備決定自己的模型要長甚麼樣子了
數據化的時候，真心覺得現在的項目太多了，資訊量太雜亂
加上這個題目又很多項目其實都是同一個東西，卻被分成好幾個項目，例如地下室空間坪數、地上的空間坪數之類的
我個人覺得這些項目根本不用分那麼細吧，分析的時候很啊雜

這時我想到前面使用過相關係數0.8，就想說乾脆暴力一點，只留下相關係數超高的項目好了哈哈
就決定drop掉所有相關係數低於0.8的項目

In [ ]:
#變數都數值化
#把要轉換的列提取出来並型成列表傳入函數中
cols = ['MSZoning', 'LotFrontage', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'SaleCondition']
df[cols] = df[cols].apply(lambda x: pd.factorize(x)[0])
df.info()

#prepare to train model!
#所有變數
var_names = df.columns.tolist()
print(var_names)
# 找出相關性大於 0.8 的變數
corr_matrix = df.corr()
high_corr_vars = []
for col in corr_matrix.columns:
    high_corr_cols = corr_matrix[corr_matrix[col] > 0.8].index.tolist()
    for high_corr_col in high_corr_cols:
        if high_corr_col != col and high_corr_col not in high_corr_vars:
            high_corr_vars.append(high_corr_col)
print(high_corr_vars)

#這些大於0.8：['1stFlrSF', 'TotalBsmtSF', 'TotRmsAbvGrd', 'GrLivArea', 'GarageArea', 'GarageCars']
X = df[['1stFlrSF', 'TotalBsmtSF', 'TotRmsAbvGrd', 'GrLivArea', 'GarageArea', 'GarageCars']]
#提出價格
y = df['SalePrice']
#重新命名變數就可以保留原有數據庫

In [ ]:
再來就引入linear regression的模型開始跑

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=54)

#using linear regression model & train
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)

#use model
predictions = reg.predict(X_test)
predictions
y_test

In [ ]:
#evaluate model
from sklearn.metrics import r2_score
r2_score(y_test, predictions)

mpl.pyplot.scatter(y_test, predictions, color='blue', alpha=0.1)

import joblib
joblib.dump(reg, 'house-20230410.pkl', compress=3)

In [ ]:
以上是app.py的檔案
以下是for_submission.py的檔案，中間使用了joblib做模型的打包

In [ ]:
import joblib
model_pretrained = joblib.load('house-20230410.pkl')
import pandas as pd
df_test = pd.read_csv("data/test.csv")
cols_to_keep = ['Id','1stFlrSF', 'TotalBsmtSF', 'TotRmsAbvGrd', 'GrLivArea', 'GarageArea', 'GarageCars']
df_test.drop(columns=[col for col in df_test.columns if col not in cols_to_keep], inplace=True)
df_test.info()

In [ ]:
進入for_submisson的檔案之後
我在老師的提醒下才知道還要把test的數據整理乾淨哈哈，感謝老師

但因為建構模型時我的想法很直接單純，就是刪去所有相關係數0.8以下的項目，所以整理數據的時候非常順利
就是該填補數據的填補數據、該drop的也不猶豫全丟掉(有種段捨離的感覺，把一堆不會常用的東西丟掉的感覺，超讚的)

唯一遇到的問題只有
start_id = df_test['Id'].min()
end_id = df_test['Id'].max()
print(start_id, end_id)
forSubmissionDF['Id'] = range(start_id, end_id+1)
這段

我一直找不到id的range，其實從df.info來看應該我的數字是正確的？但不知為何一直運行不了
最後只好套其他解法，多浪費幾條，迂迴但順利的完成這部份的程式碼

In [ ]:
df_test.loc[:, 'TotalBsmtSF'] = df_test['TotalBsmtSF'].fillna(df_test['TotalBsmtSF'].median())
df_test.loc[:, 'GarageCars'] = df_test['GarageCars'].fillna(df_test['GarageCars'].median())
df_test.loc[:, 'GarageArea'] = df_test['GarageArea'].fillna(df_test['GarageArea'].median())

prediction2 = model_pretrained.predict(df_test)
prediction2

forSubmissionDF = pd.DataFrame(columns=['Id', 'SalePrice'])
forSubmissionDF 
start_id = df_test['Id'].min()
end_id = df_test['Id'].max()
print(start_id, end_id)
forSubmissionDF['Id'] = range(start_id, end_id+1)
forSubmissionDF['SalePrice']=prediction2

forSubmissionDF.to_csv('HouseSubmission_20230410.csv', index=False)